In [170]:
pip install datasets
pip install keybert

SyntaxError: invalid syntax (215842300.py, line 1)

In [171]:
import pandas as pd
import json
import datasets
from datasets import load_dataset

In [176]:
# get the dataset only for keyphrase generation
dataset = load_dataset("midas/duc2001", "generation")

print("Samples for Keyphrase Generation")

# sample from the test split
print("Sample from test data split")
test_sample = dataset["test"][0]
print("Fields in the sample: ", [key for key in test_sample.keys()])
print("Tokenized Document: ", test_sample["document"])
print("Extractive/present Keyphrases: ", test_sample["extractive_keyphrases"])
print("Abstractive/absent Keyphrases: ", test_sample["abstractive_keyphrases"])
print("\n-----------\n")


Found cached dataset duc2001 (C:/Users/Alicia/.cache/huggingface/datasets/midas___duc2001/generation/0.0.1/7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b)


  0%|          | 0/1 [00:00<?, ?it/s]

Samples for Keyphrase Generation
Sample from test data split
Fields in the sample:  ['id', 'document', 'extractive_keyphrases', 'abstractive_keyphrases']
Tokenized Document:  ['Here', ',', 'at', 'a', 'glance', ',', 'are', 'developments', 'today', 'involving', 'the', 'crash', 'of', 'Pan', 'American', 'World', 'Airways', 'Flight', '103', 'Wednesday', 'night', 'in', 'Lockerbie', ',', 'Scotland', ',', 'that', 'killed', 'all', '259', 'people', 'aboard', 'and', 'more', 'than', '20', 'people', 'on', 'the', 'ground', ':']
Extractive/present Keyphrases:  ['pan american world airways flight 103', 'crash', 'lockerbie']
Abstractive/absent Keyphrases:  ['terrorist threats', 'widespread wreckage', 'radical palestinian faction', 'terrorist bombing', 'bomb threat', 'sabotage']

-----------



In [177]:
def drop_column(dataset):
    del dataset['abstractive_keyphrases']
    return dataset

# Para eliminar la columna que no necesitamos
dataset = dataset.map(drop_column)

Loading cached processed dataset at C:\Users\Alicia\.cache\huggingface\datasets\midas___duc2001\generation\0.0.1\7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b\cache-16ebf8679ee254b4.arrow


Hasta aquí tenemos un dataset con las 3 columnas de interés:

In [178]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'document', 'extractive_keyphrases'],
        num_rows: 308
    })
})

A continuación juntamos las palabras de 'document' porque estan separadas por comas, y para cada texto resultante implementamos keybert. Guardamos las keywords y el score de cada texto

In [179]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [180]:
keywords = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 1), stop_words=None):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keywords.append(keywords_row) # guardar información de palabras clave para cada fila


In [181]:
keywords[0] #prueba

[{'keyword': 'crash', 'score': 0.3799},
 {'keyword': 'flight', 'score': 0.3353},
 {'keyword': 'airways', 'score': 0.3078},
 {'keyword': '259', 'score': 0.2805},
 {'keyword': 'killed', 'score': 0.2766}]

In [182]:
len(keywords)

308

In [183]:
len(keywords)==len(dataset['test']) #comprovación para ver que se ha hecho para todos los textos

True

In [184]:
# Lista para almacenar las palabras clave y los puntajes
keywords_list = []

# Índices de los elementos de interés
element_indices = range(308) #o 308??

# Iterar sobre los índices y agregar las palabras clave y los puntajes a la lista
for index in element_indices:
    keywords_list.append((keywords[index][0], keywords[index][1], keywords[index][2], keywords[index][3], keywords[index][4]))

print(keywords_list)

[({'keyword': 'crash', 'score': 0.3799}, {'keyword': 'flight', 'score': 0.3353}, {'keyword': 'airways', 'score': 0.3078}, {'keyword': '259', 'score': 0.2805}, {'keyword': 'killed', 'score': 0.2766}), ({'keyword': 'tornadoes', 'score': 0.6099}, {'keyword': 'tornado', 'score': 0.5716}, {'keyword': 'thunderstorms', 'score': 0.3781}, {'keyword': 'texas', 'score': 0.3169}, {'keyword': 'storm', 'score': 0.286}), ({'keyword': 'pilots', 'score': 0.4472}, {'keyword': 'aircraft', 'score': 0.3887}, {'keyword': 'planes', 'score': 0.374}, {'keyword': 'pilot', 'score': 0.3716}, {'keyword': 'crashed', 'score': 0.3136}), ({'keyword': 'fires', 'score': 0.4183}, {'keyword': 'fire', 'score': 0.3471}, {'keyword': 'forest', 'score': 0.2806}, {'keyword': 'forests', 'score': 0.2774}, {'keyword': 'moratorium', 'score': 0.2658}), ({'keyword': 'arson', 'score': 0.5387}, {'keyword': 'fires', 'score': 0.5092}, {'keyword': 'arsonists', 'score': 0.4717}, {'keyword': 'arsonist', 'score': 0.4389}, {'keyword': 'burnin

In [185]:
# Utilizar una comprensión de lista para extraer las palabras clave
keybert_keywords = [d['keyword'] for tupla in keywords_list for d in tupla]

print(keybert_keywords)

['crash', 'flight', 'airways', '259', 'killed', 'tornadoes', 'tornado', 'thunderstorms', 'texas', 'storm', 'pilots', 'aircraft', 'planes', 'pilot', 'crashed', 'fires', 'fire', 'forest', 'forests', 'moratorium', 'arson', 'fires', 'arsonists', 'arsonist', 'burning', 'welfare', 'unemployment', 'reagan', 'jobless', 'worker', 'landings', 'landing', 'bomber', 'crashed', 'pilot', 'diamond', 'ltd', 'takeover', 'minorco', 'mining', 'crops', 'agriculture', 'agricultural', 'crop', 'drought', 'steroids', 'sprinter', 'steroid', 'issajenko', 'olympic', 'inspections', 'crashed', '737s', 'inspection', 'crash', 'eclipse', 'eclipses', 'manila', 'philippines', 'mindanao', 'aground', 'tanker', 'tankers', 'spill', 'spills', 'tornado', 'tornadoes', 'tallahassee', 'florida', 'thunderstorms', 'earthquake', 'earthquakes', 'armenia', 'deaths', 'armenian', 'insulin', 'diabetes', 'diabetics', 'pancreases', 'hormone', 'wildfires', 'hearings', 'fires', 'yellowstone', 'fire', 'wildfires', 'fires', 'firefighting', 'f

In [186]:
len(keybert_keywords)

1540

In [187]:
308*5 #es decir, 308 docs * 5 palabras clave por cada doc que nos extrae keybert Entonces la long de keybert_keywords es correcta

1540

Tal vez no nos vaya bien que la lista no esté dividida así que la pondremos en el mismo formato que tenemos las extractive_keyphrases

In [188]:
# Agrupar las palabras clave en sublistas de 5
sublists_keybert = [keybert_keywords[i:i+5] for i in range(0, len(keybert_keywords), 5)]

print(sublists_keybert)

[['crash', 'flight', 'airways', '259', 'killed'], ['tornadoes', 'tornado', 'thunderstorms', 'texas', 'storm'], ['pilots', 'aircraft', 'planes', 'pilot', 'crashed'], ['fires', 'fire', 'forest', 'forests', 'moratorium'], ['arson', 'fires', 'arsonists', 'arsonist', 'burning'], ['welfare', 'unemployment', 'reagan', 'jobless', 'worker'], ['landings', 'landing', 'bomber', 'crashed', 'pilot'], ['diamond', 'ltd', 'takeover', 'minorco', 'mining'], ['crops', 'agriculture', 'agricultural', 'crop', 'drought'], ['steroids', 'sprinter', 'steroid', 'issajenko', 'olympic'], ['inspections', 'crashed', '737s', 'inspection', 'crash'], ['eclipse', 'eclipses', 'manila', 'philippines', 'mindanao'], ['aground', 'tanker', 'tankers', 'spill', 'spills'], ['tornado', 'tornadoes', 'tallahassee', 'florida', 'thunderstorms'], ['earthquake', 'earthquakes', 'armenia', 'deaths', 'armenian'], ['insulin', 'diabetes', 'diabetics', 'pancreases', 'hormone'], ['wildfires', 'hearings', 'fires', 'yellowstone', 'fire'], ['wild

In [189]:
# Crear una lista de sublistas
extractive_keyphrases_list = []

for element in dataset['test']:
    # Acceder a la lista de extractive_keyphrases para cada elemento
    extractive_keyphrases = element['extractive_keyphrases']
    
    # Añadir la lista de extractive_keyphrases a la lista de sublistas
    extractive_keyphrases_list.append(extractive_keyphrases)
    
print(extractive_keyphrases_list)

[['pan american world airways flight 103', 'crash', 'lockerbie'], ['tornado season', 'spring thunderstorms', 'tornadoes', 'texas', 'property damage', 'tornado warning', 'tornado watches', 'disaster research'], ['crashes', 'pilots', 'bodenheim', 'training mission', 'aircraft'], ['forest fire policy', 'western fire season', 'national forests', 'fire management plans', 'natural fires', 'recommended halt', 'panel'], ['forest fires', 'criminal investigator', 'investigations', 'passing truck', 'missouri', 'arsonist', 'arson problem'], ['welfare reform', 'president reagan', 'welfare legislation', 'work requirement', 'welfare programs', 'unemployment', 'americans'], ['crash', 'training flight', 'crew members', 'pilot'], ['de beers diamond organization', 'fair trading', 'south africa', 'world diamond trading', 'defensive move', 'central selling organization', 'investigation'], ['deadening drought', 'annual agriculture department survey', 'fall corn harvest', 'higher retail food prices', 'presid

In [190]:
print(len(sublists_keybert))
print(len(extractive_keyphrases_list))


308
308


In [191]:
# Crear una lista para almacenar los recuentos
recuentos = []

# Iterar sobre las sublistas de lista1 y lista2
for sublista1, sublista2 in zip(sublists_keybert, extractive_keyphrases_list):
    # Crear un conjunto de las palabras de la sublista1
    palabras_sublista1 = set(sublista1)
    
    # Contar cuántas veces aparecen las palabras de sublista1 en sublista2
    conteos = [sublista2.count(palabra) for palabra in palabras_sublista1]
    
    # Añadir el conteo a la lista de recuentos
    recuentos.append(conteos)

print(recuentos)
    

[[0, 0, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 1, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 

In [192]:
sumas = []

# Iterar sobre las sublistas de recuentos y calcular la suma
for sublista in recuentos:
    suma = sum(sublista)
    sumas.append(suma)

print(sumas) # 308 valores, osea para cada doc cuantas de las palabras clave obtenidas con keybert se repiten en las extractive_keyphrases (las originales)

[1, 2, 2, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 3, 0, 0, 2, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1, 0, 1, 2, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 3, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 3, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0]


In [193]:
suma_total = sum(sumas)

print(suma_total) # es decir, 125 palabras se detectan como clave con keybert y efectivamente son paalabras clave 

125



Ahora podemos calcular la precisión dividiendo 125 por el total de extractive_keyphrases que hay

In [194]:
suma_extractive_keyphrases = 0
for sublista in extractive_keyphrases_list:
    suma_extractive_keyphrases += len(sublista)

print(suma_extractive_keyphrases) # no estoy segura que esto esté bien


2200


In [195]:

suma_total/suma_extractive_keyphrases #precisión  Si consideramos N-gramas d long 2 solo esto sube a 0,12...

0.056818181818181816

Para utilizar la similitud de Jaccard en la evaluación de la precisión en la extracción de palabras clave, se pueden seguir los siguientes pasos:

Obtener el conjunto de palabras clave extraídas y el conjunto de palabras clave correctas.

Calcular la intersección de ambos conjuntos, es decir, el número de palabras clave que son comunes a ambos conjuntos.

Calcular la unión de ambos conjuntos, es decir, el número total de palabras clave distintas en ambos conjuntos.

Calcular la similitud de Jaccard dividiendo el número de palabras clave en la intersección por el número de palabras clave en la unión.

In [196]:
union=((5*308)+suma_extractive_keyphrases)-suma_total # esto sería la unión (la suma del num total de palabras clave en ambos conjuntos y restar las coincidentes)
print(union)

3615


In [197]:
suma_total/union # Ngrama 2: 0,076..

0.034578146611341634

El resultado de la similitud de Jaccard es un valor entre 0 y 1, donde 0 indica que no hay similitud entre los conjuntos de palabras clave y 1 indica que son idénticos. Un valor cercano a 1 indica una alta precisión en la extracción de palabras clave.

Si miramos las extractive keyphrases, vemos que la mayoría de ellas tienen 1 o 2 N-gramas. Si ahora en vez de keywords queremos extraer kephrases de 2 palabras tenemos que cambiar el parámetro keyphrase_ngram_range:

In [198]:
keyphrases = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases.append(keywords_row) # guardar información de palabras clave para cada fila

In [199]:
keyphrases[0] #prueba

[{'keyword': 'flight 103', 'score': 0.4514},
 {'keyword': 'the crash', 'score': 0.4259},
 {'keyword': 'world airways', 'score': 0.4206},
 {'keyword': '259 people', 'score': 0.4203},
 {'keyword': 'airways flight', 'score': 0.4154}]

Después de observar que se puede modificar el tamaño de N-gramas, keybert nos permite modificar otras opciones. Una de  ellas es la Max Sum Distance, la cual coge todas las top_n combinaciones de las 2x top_n palabras que hay, y extrae las combinaciones que son menos similares entre ellas a partir de la similitud de cosinus. Ej:

In [200]:
keyphrases2 = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None, use_maxsum=True, 
                                                  nr_candidates=20, top_n=5):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases2.append(keywords_row) # guardar información de palabras clave para cada fila

In [201]:
keyphrases2[0] #prueba

[{'keyword': 'lockerbie scotland', 'score': 0.2471},
 {'keyword': 'pan american', 'score': 0.2887},
 {'keyword': 'killed all', 'score': 0.3371},
 {'keyword': 'airways flight', 'score': 0.4154},
 {'keyword': '259 people', 'score': 0.4203}]

Una vez añadidos estos parámetros, también podemos incorporar el uso de Maximal Marginal Relevance, que sirve para crear keywords/keyphrases y también se basa en la similitud de cosinus. Sirve para diversificar los resultados. Podemos probar con alta diversidad (0.7) o con baja diversidad (0.2). Ej:

In [202]:
keyphrases3 = []
for i in range(len(dataset['test'])):
    joined_text = " ".join(dataset['test'][i]['document'])
    keywords_row = []
    for keyword_info in kw_model.extract_keywords(joined_text, keyphrase_ngram_range=(1, 2), stop_words=None, use_maxsum=True, 
                                                  nr_candidates=20, top_n=5, diversity=0.7):
        keyword, score = keyword_info  # extraer palabra clave y puntuación
        keywords_row.append({'keyword': keyword, 'score': score}) # guardar información en un diccionario
    keyphrases3.append(keywords_row) # guardar información de palabras clave para cada fila

In [203]:
keyphrases3[0] #prueba, vemos que salen las mismas q antes (almenos en este caso)

[{'keyword': 'lockerbie scotland', 'score': 0.2471},
 {'keyword': 'pan american', 'score': 0.2887},
 {'keyword': 'killed all', 'score': 0.3371},
 {'keyword': 'airways flight', 'score': 0.4154},
 {'keyword': '259 people', 'score': 0.4203}]

In [204]:
keyphrases2[10]

[{'keyword': 'officials about', 'score': 0.3666},
 {'keyword': 'boeing 737s', 'score': 0.3767},
 {'keyword': 'crashed sunday', 'score': 0.3788},
 {'keyword': 'ordered inspections', 'score': 0.3837},
 {'keyword': 'engine warning', 'score': 0.3888}]

In [205]:
keyphrases3[10]

[{'keyword': 'officials about', 'score': 0.3666},
 {'keyword': 'boeing 737s', 'score': 0.3767},
 {'keyword': 'crashed sunday', 'score': 0.3788},
 {'keyword': 'ordered inspections', 'score': 0.3837},
 {'keyword': 'engine warning', 'score': 0.3888}]